In [1]:
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9

c:\Users\kuzup\OneDrive\Dokumen\GitHub\Viskom\Tugas Besar\Dataset\Fruit\yolov9


Cloning into 'yolov9'...
C:\Users\kuzup\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!curl -L -o yolov9-c.pt https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!curl -L -o yolov9-e.pt https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!curl -L -o gelan-c.pt https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!curl -L -o gelan-e.pt https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0 98.3M    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  1 98.3M    1 1098k    0     0   607k      0  0:02:45  0:00:01  0:02:44 4083k
 11 98.3M   11 11.1M    0     0  3956k      0  0:00:25  0:00:02  0:00:23 8444k
 18 98.3M   18 18.1M    0     0  4884k      0  0:00:20  0:00:03  0:00:17 8194k
 27 98.3M   27 27.0M    0     0  5751k      0  0:00:17  0:00:04  0:00:13 8458k
 36 98.3M   36 35.7M    0     0  6309k      0  0:00:15  0:00:05  0:00:10 8583k
 45 98.3M   45 44.7M    0     0  6730k      0  0:00:14  0:00:06  0:00:08 8944k
 52 98.3M   52 51.3M    0     0  6738k      0  0:00:14  0:00:07  0:00:07 8375k
 59 98.3M   59 58.5M    0     0  6809k      0  0:0

In [7]:
!pip install -r requirements.txt -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hentai 3.2.10 requires requests==2.27.1, but you have requests 2.31.0 which is incompatible.
hentai 3.2.10 requires tqdm==4.63.0, but you have tqdm 4.66.4 which is incompatible.
tiktokpy 0.9.4 requires pydantic<2.0.0,>=1.6.1, but you have pydantic 2.7.1 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: C:\Users\kuzup\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
# Prepare the dataset configuration file
data_yaml = """
train: ./Fototrain
val: ./Fototrain

nc: 2  # Number of classes
names: ['fruit', 'bomb']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)


In [12]:
!pip install torch torchvision torchaudio


                                              0.0/1.2 MB ? eta -:--:--
     --                                       0.1/1.2 MB 2.2 MB/s eta 0:00:01
     --------                                 0.2/1.2 MB 3.0 MB/s eta 0:00:01
     ----------------                         0.5/1.2 MB 3.9 MB/s eta 0:00:01
     ----------------------------             0.8/1.2 MB 4.5 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 5.0 MB/s eta 0:00:00
                                              0.0/159.8 MB ? eta -:--:--
                                              0.4/159.8 MB 8.5 MB/s eta 0:00:19
                                              0.8/159.8 MB 8.7 MB/s eta 0:00:19
                                              1.1/159.8 MB 7.6 MB/s eta 0:00:21
                                              1.5/159.8 MB 8.1 MB/s eta 0:00:20
                                              1.7/159.8 MB 7.1 MB/s eta 0:00:23
                                              2.1/159.8 MB 7.3 MB


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Prepare the model configuration file
model_yaml = """
nc: 2  # Number of classes
depth_multiple: 0.33  # Model depth multiplier
width_multiple: 0.50  # Layer channel width multiplier

anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

backbone:
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 6, BottleneckCSP, [1024, False]],  # 9
  ]

head:
  [[-1, 3, BottleneckCSP, [1024, False]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],
   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],
   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],
   [[17, 20, 23], 1, Detect, [nc, anchors]]  # Detect(P3, P4, P5)
  ]
"""
with open('models/yolo.yaml', 'w') as f:
    f.write(model_yaml)

In [5]:
import torch
import torchvision
# Train the model
!python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov9-c.pt

c:\Users\kuzup\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
python: can't open file 'c:\\Users\\kuzup\\OneDrive\\Dokumen\\GitHub\\Viskom\\Tugas Besar\\Dataset\\Fruit\\train.py': [Errno 2] No such file or directory
